<a href="https://colab.research.google.com/github/NigelWilliamUOP/vibe-coding/blob/main/exploration_of_JISC_data_for_TEF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 1. Basic setup

import pandas as pd
import numpy as np
from collections import Counter, defaultdict

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

# Then set DATA_DIR to a folder in your Drive, e.g.:
DATA_DIR = "/content/drive/MyDrive/JISC_Data"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# 2. File paths – EDIT THESE

# Set DATA_DIR to the Colab content directory for uploaded files
DATA_DIR = "/content"  # Files uploaded to Colab will typically reside here

DATA_FILE = f"{DATA_DIR}/285779_Item1_Data (1).csv"  # big Jisc staff file
LABELS_FILE = f"{DATA_DIR}/285779_Item1_Notes_and_Labelling_File (2).xlsx"

In [14]:
# 3. Read field order and labels from the Excel "Notes and Labelling" file

field_order_raw = pd.read_excel(LABELS_FILE, sheet_name="Field_Order")
field_lab_raw = pd.read_excel(LABELS_FILE, sheet_name="Field_Labelling")

# Clean up column names
field_order = field_order_raw.rename(columns={
    'Field order: 285779_Item1_Data.csv': 'field',
    'Unnamed: 1': 'description',
    'Unnamed: 2': 'format'
})

field_lab = field_lab_raw.rename(columns={
    'Field order: 285779_Item1_Data.csv': 'field_name',
    'Unnamed: 1': 'code',
    'Unnamed: 2': 'label'
})

# Drop the top header row and any empty rows
field_order = field_order[field_order['field'].notna()]
field_order = field_order[field_order['field'] != "Field"]

field_lab = field_lab[field_lab['field_name'].notna()]
field_lab = field_lab[field_lab['field_name'] != "Field Name"]

print("=== Field order (schema) ===")
display(field_order.head(20))

print("=== Example labels (codebook) ===")
display(field_lab.head(20))


=== Field order (schema) ===


,field,description,format
2,ACYEAR,Academic year,varchar(7)
3,F_UKPRN,HE provider (UKPRN) (Staff),varchar(30)
4,F_STAFFID,Staff identifier (STAFFID),varchar(30)
5,F_CONTID,Contract identifier (CONTID),varchar(20)
6,F_XACMRK01,Academic employment marker,varchar(30)
7,F_ACEMPFUN,Academic employment function (Research only/ T...,varchar(1)
8,SIGRES,Significant responsibility (2018/19 onwards),varchar(34)
9,F_STARTCON,Start date of contract (YYYY/MM),varchar(7)
10,F_ENDCON,End date of contract (YYYY/MM),varchar(7)
11,F_TERMS,Terms of employment,varchar(1)


=== Example labels (codebook) ===


,field_name,code,label
2,F_CCENTRE,01,(01) Clinical medicine
3,F_CCENTRE,02,(02) Clinical dentistry
4,F_CCENTRE,03,(03) Veterinary science
5,F_CCENTRE,04,(04) Anatomy & physiology
6,F_CCENTRE,05,(05) Nursing & paramedical studies
7,F_CCENTRE,06,(06) Health & community studies
8,F_CCENTRE,07,(07) Psychology & behavioural sciences
9,F_CCENTRE,08,(08) Pharmacy and Pharmacology
10,F_CCENTRE,09,(09) Pharmacology
11,F_CCENTRE,10,(10) Biosciences


In [15]:
# 4. Build dtype mapping from the schema

dtype_map = {}
float_fields = []

for _, row in field_order.iterrows():
    field = row['field']
    fmt = str(row['format']).lower()
    if "decimal" in fmt:
        # numeric field (FTE etc.)
        float_fields.append(field)
    else:
        # treat everything else as string initially
        dtype_map[field] = "string"

print("String fields:", len(dtype_map))
print("Decimal fields (will coerce to float):", float_fields)


String fields: 12
Decimal fields (will coerce to float): ['counter']


In [16]:
# 5. Quick sample: first N rows

N_SAMPLE = 100_000  # adjust up/down as needed

sample_df = pd.read_csv(
    DATA_FILE,
    nrows=N_SAMPLE,
    dtype=dtype_map,
    low_memory=False
)

# Convert decimal fields to float
for f in float_fields:
    if f in sample_df.columns:
        sample_df[f] = pd.to_numeric(sample_df[f], errors="coerce")

print("=== Sample shape ===")
print(sample_df.shape)

print("\n=== Sample .info() ===")
sample_df.info()

print("\n=== Head of sample ===")
display(sample_df.head())


=== Sample shape ===
(100000, 13)

=== Sample .info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ACYEAR      100000 non-null  string 
 1   F_UKPRN     100000 non-null  string 
 2   F_STAFFID   100000 non-null  string 
 3   F_CONTID    100000 non-null  string 
 4   F_XACMRK01  100000 non-null  string 
 5   F_ACEMPFUN  100000 non-null  string 
 6   SIGRES      100000 non-null  string 
 7   F_STARTCON  100000 non-null  string 
 8   F_ENDCON    100000 non-null  string 
 9   F_TERMS     100000 non-null  string 
 10  F_XMOEMP01  100000 non-null  string 
 11  F_CCENTRE   100000 non-null  string 
 12  counter     100000 non-null  float64
dtypes: float64(1), string(12)
memory usage: 9.9 MB

=== Head of sample ===


,ACYEAR,F_UKPRN,F_STAFFID,F_CONTID,F_XACMRK01,F_ACEMPFUN,SIGRES,F_STARTCON,F_ENDCON,F_TERMS,F_XMOEMP01,F_CCENTRE,counter
0,2012/13,10007157,1211590036783,10028431002,1,2,Not applicable (2017/18 and prior),2012/10,2013/03,2,2,124,0.255
1,2022/23,10007794,1411183052369,500310-341,1,1,Unknown,2020/11,9999/12,3,2,133,0.030
2,2018/19,10007141,1810530074061,62401001,2,X,Unknown,2018/10,9999/12,2,1,145,0.797
3,2020/21,10006840,2011101011952,50006929:200110,1,1,Unknown,2020/01,9999/12,1,2,102,0.100
4,2023/24,10007774,2011563861878,CASTCH-35079,1,1,Unknown,9999/12,9999/12,3,2,128,0.010


In [17]:
# 6. Build code → label dictionaries

def make_codebook(field_name: str):
    tmp = field_lab[field_lab['field_name'] == field_name]
    # Some label cells look like "(1) teaching only" – keep full text for now
    return dict(zip(tmp['code'].astype(str), tmp['label'].astype(str)))

acempfun_labels = make_codebook("F_ACEMPFUN")
xacmrk_labels = make_codebook("F_XACMRK01")

print("=== ACEMPFUN codebook ===")
for k, v in acempfun_labels.items():
    print(k, "→", v)

print("\n=== XACMRK01 codebook ===")
for k, v in xacmrk_labels.items():
    print(k, "→", v)


=== ACEMPFUN codebook ===
3 → Academic contract that is both teaching and research
9 → Academic contract that is neither teaching nor research
2 → Academic contract that is research only
1 → Academic contract that is teaching only
4 → Not an academic contract
X → Not applicable/Not required (Default code)

=== XACMRK01 codebook ===
1 → Academic contract
2 → Not an academic contract/Not applicable


In [18]:
# 6b. Attach human-readable labels to the sample

if 'F_ACEMPFUN' in sample_df.columns:
    sample_df['F_ACEMPFUN_label'] = (
        sample_df['F_ACEMPFUN'].astype(str).map(acempfun_labels)
    )

if 'F_XACMRK01' in sample_df.columns:
    sample_df['F_XACMRK01_label'] = (
        sample_df['F_XACMRK01'].astype(str).map(xacmrk_labels)
    )

print("=== Distribution of ACEMPFUN in sample ===")
display(sample_df['F_ACEMPFUN_label'].value_counts(dropna=False).head(20))

print("\n=== Distribution of academic marker in sample ===")
display(sample_df['F_XACMRK01_label'].value_counts(dropna=False))


=== Distribution of ACEMPFUN in sample ===


,count
F_ACEMPFUN_label,
Academic contract that is teaching only,28048
Not applicable/Not required (Default code),22335
Not an academic contract,19430
Academic contract that is both teaching and research,18040
Academic contract that is research only,11006
Academic contract that is neither teaching nor research,1141



=== Distribution of academic marker in sample ===


,count
F_XACMRK01_label,
Academic contract,58235
Not an academic contract/Not applicable,41765


In [19]:
# 7. Quick year / provider / function overview from the sample

key_cols = ['ACYEAR', 'F_UKPRN', 'F_ACEMPFUN', 'F_ACEMPFUN_label']

print("=== Available years in sample ===")
if 'ACYEAR' in sample_df.columns:
    display(sample_df['ACYEAR'].value_counts().sort_index())

print("\n=== Number of unique providers in sample ===")
if 'F_UKPRN' in sample_df.columns:
    print(sample_df['F_UKPRN'].nunique())
    print(sample_df['F_UKPRN'].head())

print("\n=== ACYEAR × ACEMPFUN (sample, counts) ===")
if {'ACYEAR', 'F_ACEMPFUN'}.issubset(sample_df.columns):
    ctab = (sample_df
            .pivot_table(index='ACYEAR',
                         columns='F_ACEMPFUN_label',
                         values='F_STAFFID',
                         aggfunc='count',
                         fill_value=0))
    display(ctab)


=== Available years in sample ===


,count
ACYEAR,
2012/13,8247
2013/14,8407
2014/15,8811
2015/16,8150
2016/17,8312
2017/18,8372
2018/19,8384
2019/20,7809
2020/21,7938



=== Number of unique providers in sample ===
207
0    10007157
1    10007794
2    10007141
3    10006840
4    10007774
Name: F_UKPRN, dtype: string

=== ACYEAR × ACEMPFUN (sample, counts) ===


F_ACEMPFUN_label,Academic contract that is both teaching and research,Academic contract that is neither teaching nor research,Academic contract that is research only,Academic contract that is teaching only,Not an academic contract,Not applicable/Not required (Default code)
ACYEAR,,,,,,
2012/13,1828,113,766,1884,1512,2144
2013/14,1415,125,862,2224,1536,2245
2014/15,1465,103,890,2304,1700,2349
2015/16,1442,112,880,2279,1511,1926
2016/17,1544,99,954,2216,1621,1878
2017/18,1474,86,942,2373,1614,1883
2018/19,1473,96,930,2323,1691,1871
2019/20,1385,78,917,2351,1504,1574
2020/21,1389,70,907,2471,1531,1570


In [21]:
# 8. Chunked overview across the full dataset

CHUNK_SIZE = 250_000  # adjust depending on Colab RAM

year_counts = Counter()
acempfun_counts = Counter()
year_acempfun_counts = Counter()
provider_year_fte = defaultdict(lambda: {"fte_total": 0.0,
                                         "fte_teach_only": 0.0,
                                         "fte_academic": 0.0})

chunk_idx = 0

for chunk in pd.read_csv(
    DATA_FILE,
    dtype=dtype_map,
    low_memory=False,
    chunksize=CHUNK_SIZE,
    on_bad_lines='skip' # Added to skip malformed rows
):
    chunk_idx += 1
    print(f"Processing chunk {chunk_idx}...")

    # Convert decimal 'counter' to float (FTE)
    for f in float_fields:
        if f in chunk.columns:
            chunk[f] = pd.to_numeric(chunk[f], errors="coerce")

    # Basic safety: drop rows without ACYEAR or provider
    if 'ACYEAR' not in chunk.columns or 'F_UKPRN' not in chunk.columns:
        raise ValueError("Expected ACYEAR and F_UKPRN columns not found.")

    # Update year and ACEMPFUN counts
    year_counts.update(chunk['ACYEAR'].value_counts().to_dict())

    if 'F_ACEMPFUN' in chunk.columns:
        acempfun_counts.update(chunk['F_ACEMPFUN'].value_counts().to_dict())
        # ACYEAR × ACEMPFUN
        grp = (
            chunk
            .groupby(['ACYEAR', 'F_ACEMPFUN'])
            .size()
            .to_dict()
        )
        year_acempfun_counts.update(grp)

    # Build provider-year FTE totals for academic staff
    # Academic contracts only: XACMRK01 = '1' (per codebook)
    if {'F_XACMRK01', 'counter'}.issubset(chunk.columns):
        is_academic = chunk['F_XACMRK01'] == '1'
        ac_chunk = chunk[is_academic].copy()

        # Teaching-only flag: ACEMPFUN = '1'
        teach_only_mask = ac_chunk['F_ACEMPFUN'] == '1'

        # Group by provider-year
        grp_total = (
            ac_chunk
            .groupby(['ACYEAR', 'F_UKPRN'])['counter']
            .sum()
        )

        grp_teach_only = (
            ac_chunk[teach_only_mask]
            .groupby(['ACYEAR', 'F_UKPRN'])['counter']
            .sum()
        )

        for (year, ukprn), fte in grp_total.items():
            key = (year, ukprn)
            provider_year_fte[key]['fte_total'] += float(fte)
            provider_year_fte[key]['fte_academic'] += float(fte)

        for (year, ukprn), fte in grp_teach_only.items():
            key = (year, ukprn)
            provider_year_fte[key]['fte_teach_only'] += float(fte)

print("Done.")

Processing chunk 1...
Processing chunk 2...
Done.


In [22]:
# 8b. Summaries from the counters

print("=== Staff counts by ACYEAR (all staff, row counts) ===")
display(pd.Series(year_counts).sort_index())

print("\n=== Staff counts by ACEMPFUN code (row counts) ===")
acempfun_series = pd.Series(acempfun_counts).sort_index()
display(acempfun_series)

print("\n=== ACEMPFUN codebook recap ===")
for code in acempfun_series.index:
    print(code, "→", acempfun_labels.get(str(code), "?"))


=== Staff counts by ACYEAR (all staff, row counts) ===


,0
2012/13,40148
2013/14,41214
2014/15,42444
2015/16,39475
2016/17,39968
2017/18,40539
2018/19,40768
2019/20,38104
2020/21,38422
2021/22,42013



=== Staff counts by ACEMPFUN code (row counts) ===


,0
1,135763
2,53095
3,86781
4,94574
9,5693
X,108574



=== ACEMPFUN codebook recap ===
1 → Academic contract that is teaching only
2 → Academic contract that is research only
3 → Academic contract that is both teaching and research
4 → Not an academic contract
9 → Academic contract that is neither teaching nor research
X → Not applicable/Not required (Default code)


In [23]:
# 8c. Provider-year FTE table with teaching-only share

records = []
for (year, ukprn), vals in provider_year_fte.items():
    fte_total = vals['fte_total']
    fte_teach = vals['fte_teach_only']
    share_teach_only = fte_teach / fte_total if fte_total > 0 else np.nan
    records.append({
        'ACYEAR': year,
        'F_UKPRN': ukprn,
        'fte_academic_total': fte_total,
        'fte_teaching_only': fte_teach,
        'share_teaching_only': share_teach_only
    })

provider_year_df = pd.DataFrame.from_records(records)

print("=== Provider-year FTE table (head) ===")
display(provider_year_df.head())

print("\n=== Years in provider-year table ===")
display(provider_year_df['ACYEAR'].value_counts().sort_index())

print("\n=== Basic distribution of teaching-only share ===")
display(provider_year_df['share_teaching_only'].describe())


=== Provider-year FTE table (head) ===


,ACYEAR,F_UKPRN,fte_academic_total,fte_teaching_only,share_teaching_only
0,2012/13,10000291,53.86800,12.65800,0.234982
1,2012/13,10000385,10.88300,3.34900,0.307728
2,2012/13,10000571,20.77800,2.71400,0.130619
3,2012/13,10000712,22.50950,0.00000,0.000000
4,2012/13,10000824,34.22944,4.01488,0.117293



=== Years in provider-year table ===


,count
ACYEAR,
2012/13,162
2013/14,162
2014/15,163
2015/16,161
2016/17,162
2017/18,163
2018/19,166
2019/20,186
2020/21,199



=== Basic distribution of teaching-only share ===


,share_teaching_only
count,2121.000000
mean,0.307268
std,0.288098
min,0.000000
25%,0.104780
50%,0.205128
75%,0.409318
max,1.000000


In [24]:
import os
OUT_DIR = "/content/jisc_staff_outputs"
os.makedirs(OUT_DIR, exist_ok=True)
print("Saving outputs to:", OUT_DIR)


Saving outputs to: /content/jisc_staff_outputs


In [25]:
# Year-level row counts from year_counts Counter
year_counts_df = (
    pd.Series(year_counts, name="row_count")
      .reset_index()
      .rename(columns={"index": "ACYEAR"})
      .sort_values("ACYEAR")
)

print("=== Year counts ===")
display(year_counts_df)

year_counts_path = os.path.join(OUT_DIR, "year_counts.csv")
year_counts_df.to_csv(year_counts_path, index=False)
print("Saved:", year_counts_path)


=== Year counts ===


,ACYEAR,row_count
8,2012/13,40148
2,2013/14,41214
0,2014/15,42444
9,2015/16,39475
7,2016/17,39968
3,2017/18,40539
6,2018/19,40768
11,2019/20,38104
10,2020/21,38422
1,2021/22,42013


Saved: /content/jisc_staff_outputs/year_counts.csv


In [26]:
# ACEMPFUN counts + labels
acempfun_counts_df = (
    pd.Series(acempfun_counts, name="row_count")
      .reset_index()
      .rename(columns={"index": "F_ACEMPFUN"})
      .sort_values("F_ACEMPFUN")
)

# Attach human labels where available
acempfun_counts_df["F_ACEMPFUN_label"] = (
    acempfun_counts_df["F_ACEMPFUN"]
      .astype(str)
      .map(acempfun_labels)
)

print("=== ACEMPFUN counts ===")
display(acempfun_counts_df)

acempfun_counts_path = os.path.join(OUT_DIR, "acempfun_counts.csv")
acempfun_counts_df.to_csv(acempfun_counts_path, index=False)
print("Saved:", acempfun_counts_path)


=== ACEMPFUN counts ===


,F_ACEMPFUN,row_count,F_ACEMPFUN_label
0,1,135763,Academic contract that is teaching only
4,2,53095,Academic contract that is research only
3,3,86781,Academic contract that is both teaching and re...
2,4,94574,Not an academic contract
5,9,5693,Academic contract that is neither teaching nor...
1,X,108574,Not applicable/Not required (Default code)


Saved: /content/jisc_staff_outputs/acempfun_counts.csv


In [27]:
# Convert year_acempfun_counts Counter into a tidy DataFrame
rows = []
for (year, acfun), count in year_acempfun_counts.items():
    rows.append({
        "ACYEAR": year,
        "F_ACEMPFUN": acfun,
        "row_count": count
    })

year_acempfun_df = pd.DataFrame(rows)

# Add labels
year_acempfun_df["F_ACEMPFUN_label"] = (
    year_acempfun_df["F_ACEMPFUN"]
      .astype(str)
      .map(acempfun_labels)
)

# Sort nicely
year_acempfun_df = year_acempfun_df.sort_values(["ACYEAR", "F_ACEMPFUN"])

print("=== ACYEAR × ACEMPFUN (row counts) ===")
display(year_acempfun_df.head(30))

year_acempfun_path = os.path.join(OUT_DIR, "year_acempfun_counts.csv")
year_acempfun_df.to_csv(year_acempfun_path, index=False)
print("Saved:", year_acempfun_path)


=== ACYEAR × ACEMPFUN (row counts) ===


,ACYEAR,F_ACEMPFUN,row_count,F_ACEMPFUN_label
0,2012/13,1,8848,Academic contract that is teaching only
1,2012/13,2,3901,Academic contract that is research only
2,2012/13,3,8734,Academic contract that is both teaching and re...
3,2012/13,4,7423,Not an academic contract
4,2012/13,9,643,Academic contract that is neither teaching nor...
5,2012/13,X,10599,Not applicable/Not required (Default code)
6,2013/14,1,10830,Academic contract that is teaching only
7,2013/14,2,4131,Academic contract that is research only
8,2013/14,3,6916,Academic contract that is both teaching and re...
9,2013/14,4,7663,Not an academic contract


Saved: /content/jisc_staff_outputs/year_acempfun_counts.csv


In [28]:
print("=== Provider-year FTE panel (preview) ===")
display(provider_year_df.head())

provider_year_path = os.path.join(OUT_DIR, "provider_year_fte_teachshare.csv")
provider_year_df.to_csv(provider_year_path, index=False)
print("Saved:", provider_year_path)


=== Provider-year FTE panel (preview) ===


,ACYEAR,F_UKPRN,fte_academic_total,fte_teaching_only,share_teaching_only
0,2012/13,10000291,53.86800,12.65800,0.234982
1,2012/13,10000385,10.88300,3.34900,0.307728
2,2012/13,10000571,20.77800,2.71400,0.130619
3,2012/13,10000712,22.50950,0.00000,0.000000
4,2012/13,10000824,34.22944,4.01488,0.117293


Saved: /content/jisc_staff_outputs/provider_year_fte_teachshare.csv


In [29]:
# Example: keep only 2012/13 onwards
mask = provider_year_df["ACYEAR"].astype(str) >= "2012/13"
provider_year_trim = provider_year_df[mask].copy()

provider_year_trim_path = os.path.join(OUT_DIR, "provider_year_fte_teachshare_2012on.csv")
provider_year_trim.to_csv(provider_year_trim_path, index=False)
print("Saved:", provider_year_trim_path)


Saved: /content/jisc_staff_outputs/provider_year_fte_teachshare_2012on.csv


In [30]:
# ACEMPFUN codebook
acempfun_codebook_df = (
    field_lab[field_lab["field_name"] == "F_ACEMPFUN"]
      .rename(columns={"code": "F_ACEMPFUN", "label": "F_ACEMPFUN_label"})
      [["F_ACEMPFUN", "F_ACEMPFUN_label"]]
      .dropna()
)

acempfun_codebook_path = os.path.join(OUT_DIR, "codebook_F_ACEMPFUN.csv")
acempfun_codebook_df.to_csv(acempfun_codebook_path, index=False)
print("Saved:", acempfun_codebook_path)

# XACMRK01 codebook
xacmrk_codebook_df = (
    field_lab[field_lab["field_name"] == "F_XACMRK01"]
      .rename(columns={"code": "F_XACMRK01", "label": "F_XACMRK01_label"})
      [["F_XACMRK01", "F_XACMRK01_label"]]
      .dropna()
)

xacmrk_codebook_path = os.path.join(OUT_DIR, "codebook_F_XACMRK01.csv")
xacmrk_codebook_df.to_csv(xacmrk_codebook_path, index=False)
print("Saved:", xacmrk_codebook_path)


Saved: /content/jisc_staff_outputs/codebook_F_ACEMPFUN.csv
Saved: /content/jisc_staff_outputs/codebook_F_XACMRK01.csv
